### Chapter 1

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('../data/default_of_credit_card_clients__courseware_version_1_21_19.xls')

##### The business problem:
Our client is a credit card company. They have brought us a dataset that includes some demographics and recent financial data (the past six months) for a sample of 30,000 of their account holders. This data is at the credit account level; in other words, there is one row for each account. Rows are labeled by whether in the next month after six month historical data period, an account owner has defaulted, or in other words, failed to make the minimum payment.

##### Goal:
Goal is to develop a predictive model for whether an account will default next month, given demographics and historical data.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  object
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  int64 
 3   EDUCATION                   30000 non-null  int64 
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_1                       30000 non-null  object
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

#### Data validation

In [4]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
# We expect 30,000 samples. Le1t's check if there really are 30,000 unique sample IDs
df['ID'].nunique()

29687

In [6]:
df.shape

(30000, 25)

There are 30,000 samples out of which only 29,687 are unique.

In [7]:
id_counts = df['ID'].value_counts()
id_counts.head()

ID
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
75938fec-e5ec    2
2a793ecf-05c6    2
Name: count, dtype: int64

In [8]:
id_counts.value_counts()

count
1    29374
2      313
Name: count, dtype: int64

As we can see nearly all IDs occur only once and some of them occur twice. Now, let's mask the duplicates

In [9]:
dupe_mask = id_counts == 2
dupe_mask[0:5]

ID
89f8f447-fca8    True
7c9b7473-cc2f    True
90330d02-82d9    True
75938fec-e5ec    True
2a793ecf-05c6    True
Name: count, dtype: bool

In [10]:
# Access duplicate IDs
dupe_ids = id_counts.index[dupe_mask]
dupe_ids = list(dupe_ids)
len(dupe_ids)

313

In [15]:
dupe_mask

ID
89f8f447-fca8     True
7c9b7473-cc2f     True
90330d02-82d9     True
75938fec-e5ec     True
2a793ecf-05c6     True
                 ...  
bd737997-0eb0    False
6d4101a3-72cb    False
ffba3018-ce71    False
df11c637-f8c2    False
ef4d40c5-8697    False
Name: count, Length: 29687, dtype: bool

Now after finding duplicate IDs, let's explore what they are

In [32]:
df.loc[df['ID'].isin(dupe_ids)].head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
141,26bde6da-f148,180000,1,2,2,28,-1,-1,-1,-1,...,332,416,416,0,416,332,500,3500,832,0
196,42009c72-651f,150000,2,2,1,34,-2,-2,-2,-2,...,116,0,1500,0,0,116,0,1500,0,0
241,26bde6da-f148,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,42009c72-651f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323,73ed1381-3a3f,230000,2,2,1,28,0,0,0,-2,...,2100,767,6173,1710,1100,1000,767,6173,1000,1


We can see that of the two duplicate rows only one is valid. As the other one is all zeroes.

In [40]:
df_zero_mask = df == 0
feature_zero_mask = df_zero_mask.iloc[:, 1:].all(axis=1)
sum(feature_zero_mask)  # The number of rows with all zeroes except for ID

315

The number of duplicate rows is 313 and the number of rows with all zeroes is 315, therefore if we delete all of these rows we may get rid of the duplicate ID problem.

In [47]:
df_clean_1 = df.loc[~feature_zero_mask].copy()  # Create a clean copy of the original data
df_clean_1.shape[0] == df_clean_1['ID'].nunique()  # Check if all rows have unique IDs

True